# <span style="color: blue;">Intro</span>

The first part of the analysis is to see what happens when ABRicate is run with both the Resfinder and CARD database to identify the extent of difference in how each method identifies "TRANSMISSIBLE RESISTANCE GENES"
Note by the fact we are only looking at "TRANSMISSIBLE RESISTNACE GENES" CARD and Resfinder dbs should have all necessary genes in them

**For two programs to identify the same thing they have to identify everything identically at the protein level**

### Setup 

**Dependencies**
* Python 3
* Biopython

**Inputs - Resistance Databases**
* resfinder_20191001 (Primary)
* card_20191023
* resfinder_20180122
* resfinder_20170126




In [1]:
# Importing modules and looking at local files

# Python inbuilt
import json
import csv
import subprocess
import os

# Extra downloaded
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Data handling
import numpy as np
import pandas as pd

# File Structure
%ls

card_20191023/                      resfinder_20180122/
card_20191023_formatted.fasta       resfinder_20190122/
card_20191023_link.csv              resfinder_20191001/
db_comparison.csv                   resfinder_20191001_ami/
db_comparison_full.json             resfinder_20191001_blm/
db_comparison_result_tarballs/      resfinder_20191001_formatted.fasta
db_related.ipynb                    resfinder_20191001_link.csv
formatting_old_resfinder_dbs.ipynb  resfinder_20191001_qui/
resfinder.fasta                     resfinder_20191001_sul/
resfinder_20170126/                 resfinder_20191001_tri/


In [2]:
# First we PARSE the databases 
# CARD
card_db = "card_20191023/card-data/card.json"
with open(card_db) as f:
    card_db = json.load(f)

    

## <span style="color: blue;">Formatting and comparing the two databases</span>


So for comparing CARD with Resfinder results, were aiming to identify transmissible proteins from each
In card this approximately equates to the **protein homolog model**.

This process has several steps
1. For each database, create a csv which contains an easily accesible set of information for comparison
- Note for working out whether there is a translation link, we translate with translation table 11, then disregard the first element and remove any trailing stop codon sequence.
- Looking at the remaining sequence, if they match then there is a link, if not no link
- If no link leave blank, and each DNA with no link will be put next to its directly translated protein
2. As part of making a database each element will get a new identifier which will be used to make sure all programs can process them the same way
- e.g. cardnewid_x 
2. Match these CSVs according to protein sequence




#### The card CSV will have the following headers

1. prot_seq
2. dna_seq
3. card_name
4. card_newname
5. aro_id
6. translation_link

#### The resfinder CSV will have the following headers

1. prot_seq
2. dna_seq
3. resfinder_name
4. resfinder_newname


#### DATABASE QUALITY CONTROL

For each database, we need to create a unique identifier for elements
This based on its DNA sequence only
This in effect will remove a sequence which has the same DNA sequence but different name in any given database

e.g. supposed the following sequence AACTTGCTA was called both gene1 and gene2 in the formated databases it will only be called newgene1

Duplicates names for different sequences will be asigned new names. The aim of this approach is to retain the databases in as close to original format as possible , while making up to date databases readable for each of the 4 programs

##### Resfinder 1st October 2019 release

**Note the resfinder database is produced by concatinating each of the specific antibiotic databases, this results in some duplicates which we remove, (i.e. the genes which affect more than one type of antibiotic, (for example quinolone resistance causing aac variant)**

###### Removed variants below

So the duplicate names we will have censored using this method
1. "blaOXA-347_1_JN086160" , same sequence as "blaOXA-347_1_ACWG01000053" (2 refs)
2. "blaZ_129_CP003194", same sequence as "blaZ_125_CP003194" (curation)
3. "blaIMP-58_1_KU647281",  same sequence as "blaIMP-58_1_KU647281" (duplciate)
4. "blaCTX-M-63_1_AB205197", same sequence as "blaCTX-M-63_1_EU660216" (2 refs)
5. "blaCMY-110_1_AB872957", same sequence as "blaCMY-110_1_AB872957" (duplicate)
6. "blaCMY-104_1_KF150216", same sequence as "blaCMY-104_1_KF150216" (duplicate)
7. "blaACC-4_2_EF504260", same sequence as "blaACC-4_1_GU256641" (2 refs)
8. "blaSHV-36_1_AF467947", same sequence as "blaSHV-36_1_AF467947" (duplicate)
9. "blaOXA-60_1_AF525303", same sequence as "blaOXA-60_1_AF525303" (duplicate)
10. "blaFRI-1_1_KT192551", same sequence as "blaFRI-1_1_KT192551" (duplicate)
11. "cfr_1_AM408573", same sequence as "cfr_1_AM408573" (duplicate)
12. "cfr_2_AJ879565", same sequence as "cfr_2_AJ879565" (duplicate)
13. "cfr(B)_3_KR610408", same sequence as "cfr(B)_3_KR610408" (duplicate)
14. aac(6')-Ib-cr_1_DQ303918, same sequence as "aac(6')-Ib-cr_1_DQ303918" (duplicate)
15. aac(6')-Ib-cr_2_EF636461 , same sequence as "aac(6')-Ib-cr_2_EF636461" (duplicate)
16. dfrA22_3_FM957884, same sequence as dfrA33_1_FM957884 (curation)

##### CARD 23rd October 2019 release

This database is more complex in structure, but to attempt to produce a similar set of genes (i.e. just the transmissible genes) we only look at the genes which apply to the "protein homolog model"

In this database, there are several things to note which will be important as we compare the database:
* Each element has at least one DNA and protein sequence
* all of the DNA elements do translate to produce the related protein , but there is an issue with wether or not sequences include a stop codon and how the start codon is translated. Also some are not in the correct frame
* Also 1 sequence (Erm(44)v) has 2 sequences. This is not relavent to this study as it focusses on E. coli whereas this gene is only seen in S. saprophyticus and encodes macrolide resistance (to which E. coli is innately resistant)
* For simplicity here, we will just take the DNA sequences from the protein homolog CARD.
* Otherwise sequences in this database are all unique (see Resfinder Issues above)


#### DATABASE PREPARATION CHOICES

This is any specific parameters we have chosen in preparing databases

**ARIBA -> Non coding only**

In [3]:
# CARD

# This code makes the formatted card db ("card_20191023_formatted.fasta") 
# and produces a key for the identifiers in the fasta file "card_20191023_link.csv"
# And finally produces a dictionary which we use downstream "card_20191023_link.csv"

card_formatted_no = 1

card_dict = {}

card_recs = []

with open("card_20191023_link.csv", "w") as f:
    writer = csv.writer(f)
    for key in card_db.keys():
        try:
            # First we extract relavent data
            if card_db[key]['model_type'] == "protein homolog model":
                aro_id = card_db[key]['ARO_id']
                name = card_db[key]["ARO_name"]
                for k in card_db[key]['model_sequences']['sequence'].values():
                    card_prot = k['protein_sequence']['sequence']
                    card_dna = k['dna_sequence']['sequence']
                # This way we automatically keep the second sequence
                # Then we rename things so that each program formatts them properly.
                new_id = "cardnewid_{0}" .format(card_formatted_no)
                card_formatted_no += 1
                card_seq = Seq(card_dna)
                card_rec = SeqRecord(card_seq, id=new_id, description="")
                card_recs.append(card_rec)
                # Now we want to create some stuff for a linking database to compare the two
                orig_id = name + "_ARO"+ aro_id
                card_dict[new_id] = {"orig_id":orig_id, 
                                    "dna_seq":card_dna,
                                    "prot_seq":card_prot}
                writer.writerow([orig_id, new_id, str(card_seq)])
        except:
            # This line removes the 1 entry which just has a description of the database
            pass

SeqIO.write(card_recs, "card_20191023_formatted.fasta", "fasta")

2617

In [4]:
# Resfinder

# Likewise for the resfinder database
# The formatted fasta "resfinder_20191001_formatted.fasta"
# link for ids "resfinder_20191001_link.csv"
# Dictionary for downstream analysis resfinder_db

# Firstly, you concatenate and read in the database

##############################################################################
# NOTE THIS NEEDS TO BE FIXED BEFORE WE PUT THIS UP FORMALLY ONLINE
if os.name == "posix":
    subprocess.check_call("cat resfinder_20191001/*.fsa > resfinder.fasta", shell=True)
else:
    pass
resfinder_initdb = SeqIO.parse("resfinder_20191001/resfinder.fasta", "fasta")
##############################################################################

# Then you assign a new unique name to each element of the database
# Note some elements are removed (see above) to ensure each DNA sequence only has one name
# The way things are linked are put into a resfinder_20191001_link.csv file
# The final database is then written into a resfinder_20191001_formatted.fasta
resfinder_db = {}
identified_seqs = []
newid = 0
for k in resfinder_initdb:
    if str(k.seq) not in identified_seqs:
        newid += 1
        identified_seqs.append(str(k.seq))
        resfinder_db["resfindernewid_{0}" .format(newid)] = k
out_recs = []
with open("resfinder_20191001_link.csv", "w") as f:
    writer = csv.writer(f, delimiter = ",")
    for k in resfinder_db:
        writer.writerow([k, resfinder_db[k].id, str(resfinder_db[k].seq)])
        k_id = k
        k_desc = ""
        k_seq = resfinder_db[k].seq
        k_rec = SeqRecord(k_seq, id=k_id, description=k_desc)
        out_recs.append(k_rec)

SeqIO.write(out_recs, "resfinder_20191001_formatted.fasta", "fasta")

3079

## <span style="color: blue;">Next comes how to format the databases </span>

This code basically goes through the exact commands we use to format the databases for each program. Note for each one we using singularity images which contain each program. Your actual locations of some script (e.g. the SRST2 specific ones)  may vary

As an additional side point this database then needs to be formatted to meet ABRicate's structure
The script below does this

~~~
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

base_db = SeqIO.parse("<db_file>","fasta")
abricate_recs = []

for k in base_db:
    k_seq = k.seq
    k_id = "resfinderformatted~~~{0}~~~00{1}".format(k.id, "{:04d}" .format(int(k.id.split("_")[-1])))
    k_rec = SeqRecord(k_seq, id=k_id, description="")
    abricate_recs.append(k_rec)
SeqIO.write(abricate_recs, "sequences", "fasta")
~~~


##### Each of the following illustrates the steps required to prep the database

**preparing the ABRicate_db**

~~~
python abricate_format.py 
mkdir abricate_db
mv sequences abricate_db/
cd abricate_db
makeblastdb -in sequences -title <db_title> -dbtype nucl -hash_index
cd ..
~~~
**preparing the ARIBA db**
~~~
ariba prepareref --all_coding no -f <db_file> ariba_db
~~~

**Preparing the KmerResistance db**
** Note for KmerResistance, there is an additional file**
### add to this text ### should primarily been in the supplements

The exception to this was that KmerResistance requires a “bacteria.fsa” FASTA file of all complete genomes in NCBI’s RefSeq database to filter low-coverage matches. As of writing, the version used by KmerResistance’s authors was not publicly available, however we attempted mitigate this using two alterations; replacing the file using a FASTA containing all complete genomes as identified by Centrifuge-download[14], and applying an average depth of coverage cut-off of >5x to KmerResistance results. For ABRicate (which uses BLASTn to search assemblies), assemblies were produced using SPAdes[8] run with default parameters. 

~~~
mkdir kmerres_db
mv bacteria.fsa kmerres_db
cp <db_file> kmerres_db/
cd kmerres_db
kma index -i bacteria.fsa -o bacteria -Sparse ATG
mv <db_file> kmerres_fasta.fa
kma_index -i kmerres_fasta.fa -o kmerres_fasta
cd ..
~~~

**Preparing the SRST2 db**
~~~
mkdir srst2_db
cd srst2_db/
mv <db_file> rawseqs.fasta
cd-hit-est -i rawseqs.fasta -o rawseqs_cdhit90 -d 0 > rawseqs_cdhit90.stdout
python /srst2/database_clustering/cdhit_to_csv.py --cluster_file rawseqs_cdhit90.clstr --infasta rawseqs.fasta --outfile rawseqs_clustered.csv
python /srst2/database_clustering/csv_to_gene_db.py -t rawseqs_clustered.csv -o seqs_clustered.fasta -f rawseqs.fasta -c 4
cd ..
~~~

** A similar method is used for preparing the card and legacy resfinder databases **





## <span style="color: blue;">Comparing the databases</span>

Next step we want to compare the two databases
The resfinder database is now encapsulated in the resfinder_db dictionary
The card database is now encapsuled in card_dict

**Now to compare these two we are going to do the following.**
1. translate resfinder_db sequences
2. compare the protein sequences from the two databases. [note this requires some wiggle room re first and last amino-acids, see above]
3. give each unique protein sequence a unique id.
4. Identify whether DNA variants are the same or different

**Note all elements of each database are disctinct (we have removed duplicates as part of formatting)**


In [5]:

# First we start with a function to compare two proteins with "wiggle room"
def compare_2_prots(s1, s2):
    # Now as there is a bit of a wiggle around what sequence is at 
    s1_poss = [s1,s1[1:],s1[:-1],s1[1:-1]]
    s2_poss = [s2,s2[1:],s2[:-1],s2[1:-1]]
    linked = False
    for k in s1_poss:
        if k in s2_poss:
            linked = True
    return linked

# Next lets get all of the proteins from both the CARD and Resfinder databases
resfinder_prots = {}
for k in resfinder_db:
    resfinder_prots[k] = str(resfinder_db[k].seq.translate(11))

card_prots = {}
for k in card_dict:
    card_prots[k] = card_dict[k]['prot_seq']


# Now we combine these into a list of individual proteins
prot_list = []
for k in resfinder_prots:
    seen_before  = False
    for j in prot_list:
        if compare_2_prots(resfinder_prots[k], j) == True:
            seen_before = True
    if seen_before == False:
        prot_list.append(resfinder_prots[k])
    
for k in card_prots:
    seen_before = False
    for j in prot_list:
        if compare_2_prots(card_prots[k], j) == True:
            seen_before = True
    if seen_before == False:
        prot_list.append(card_prots[k])

#Turning these into a dictionary
prots_full = {}
prot_no = 1
for k in prot_list:
    prot_id = "uprotein_{0}".format(prot_no)
    prots_full[prot_id] = k
    prot_no += 1

# Finally we want to make an output which combines all of these.
# We will produce two however, 
# 1 which only contains the simple names linking (and official names) (CSV) and 1 which contains all of the details about the sequence (JSON)

final_dict = {}
with open("db_comparison.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["prot_id", 'card_fids','card_oids', 'resfinder_fids', 'resfinder_oids'])
    for k in prots_full:
        final_dict[k] ={"card":{}, "resfinder":{}, "protein":prots_full[k]}
        k_card_ids = []
        k_resfinder_ids = []
        for j in card_prots:
            if compare_2_prots(prots_full[k], card_prots[j]) == True:
                k_card_ids.append(j)
        k_card_origid = [card_dict[j]['orig_id'] for j in k_card_ids]
        for j in k_card_ids:
            final_dict[k]['card'][j] = card_dict[j]
        for j in resfinder_prots:
            if compare_2_prots(prots_full[k], resfinder_prots[j]) == True:
                k_resfinder_ids.append(j)
        for j in k_resfinder_ids:
            final_dict[k]['resfinder'][j] = {resfinder_db[j].id:str(resfinder_db[j].seq)}
        k_resfinder_origid = [resfinder_db[j].id for j in k_resfinder_ids]
        csv_line = [k, ":".join(k_card_ids),":".join(k_card_origid), 
                   ":".join(k_resfinder_ids), ":".join(k_resfinder_origid)]
        writer.writerow(csv_line)

# NOTE FOR ME, see sublime_text on problems with 3 SHV sequences. I will currently plough on regardless 
        
with open("db_comparison_full.json", "w") as f:
    x = json.dumps(final_dict)
    json.dump(x, f) 


/Users/timdavies/anaconda/lib/python3.6/site-packages/Bio/Seq.py:2715: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


## <span style="color: blue;">Comparing Resfinder and CARD output</span>

Note I will only investigate differences between the two that I encounter in my data., 
so the next step will be to compare the output between CARD and Resfinder.

This bit needs to be updated, given the figure I intend to produce? compare all full databases.


In [14]:
#############################################################
#               LOADING THE DATA                            #
#############################################################

# Note during the covid outbreak I have created a temporary file structure to enable analysis of files
# This option (if true) enables using it
temp = False
# THIS CHOICE WILL BE DELETED PRIOR TO PRE_PRINT publication

result_dict = {}

linking_csv = pd.read_csv("db_comparison.csv", index_col=0).fillna("")
card_linkdict = {}
card_names = {}
for k in linking_csv.index:
    for j in range(len(linking_csv.loc[k].card_fids.split(":"))):
        j_fid = linking_csv.loc[k].card_fids.split(":")[j]
        j_oid = linking_csv.loc[k].card_oids.split(":")[j]        
        card_linkdict[j_fid] = k
        card_names[k] = j_oid
        
resfinder_linkdict = {}
resfinder_names = {}
for k in linking_csv.index:
    for j in range(len(linking_csv.loc[k].resfinder_fids.split(":"))):
        j_fid = linking_csv.loc[k].resfinder_fids.split(":")[j]
        j_oid = linking_csv.loc[k].resfinder_oids.split(":")[j]
        resfinder_linkdict[j_fid]=k
        resfinder_names[k]=j_oid

card_ftranslate = pd.read_csv("card_20191023/card_20191023_link.csv", header=None, index_col = 1)
card_ftranslate = {k: card_ftranslate.loc[k][0] for k in card_ftranslate.index}
card_btranslate = pd.read_csv("card_20191023/card_20191023_link.csv", header=None, index_col = 0)
card_btranslate = {k: card_btranslate.loc[k][1] for k in card_btranslate.index}
resfinder_ftranslate = pd.read_csv("resfinder_20191001/resfinder_20191001_link.csv", header=None, index_col = 1)
resfinder_ftranslate = {k: resfinder_ftranslate.loc[k][0] for k in resfinder_ftranslate.index}
resfinder_btranslate = pd.read_csv("resfinder_20191001/resfinder_20191001_link.csv", header=None, index_col = 0)
resfinder_btranslate = {k: resfinder_btranslate.loc[k][1] for k in resfinder_btranslate.index}

card_dir = "db_comparison_result_tarballs/card_results/"
card_reports = [os.path.join(root, f) for root, dirs, files in os.walk(card_dir) for f in files]
rf_dir = "db_comparison_result_tarballs/resfinder_results/"
rf_reports = [os.path.join(root, f) for root, dirs, files in os.walk(rf_dir) for f in files]
guuids = [k.split("/")[-1].split("_")[0] for k in rf_reports if "summary" not in k]
result_dict = {k:{"card":"", "resfinder":""} for k in guuids}
for k in result_dict:
    card_fl = [f for f in card_reports if k in f]
    assert len(card_fl) == 1
    card_fl = card_fl[0]
    resfinder_fl = [f for f in rf_reports if k in f]
    assert len(resfinder_fl) == 1 
    resfinder_fl = resfinder_fl[0]
    result_dict[k]["card"] = card_fl
    result_dict[k]["resfinder"] = resfinder_fl

### DELETE THIS SECTION FOR pre-print publication

# this code essentially produces a dictionary where keys are guuids and then within that 
# there is a card and a resfinder reports

if temp == True:
    card_dir = "../../../result_tarballs/pl_comp_data/db_comparison_result_tarballs/card_results"
    card_reports = [os.path.join(root, f) for root, dirs, files in os.walk(card_dir) for f in files]
    rf_dir = "../../../result_tarballs/pl_comp_data/db_comparison_result_tarballs/resfinder_results"
    rf_reports = [os.path.join(root, f) for root, dirs, files in os.walk(rf_dir) for f in files]
    guuids = [k.split("\\")[-1].split("_")[0] for k in rf_reports if "summary" not in k]
    result_dict = {k:{"card":"", "resfinder":""} for k in guuids}
    for k in result_dict:
        card_fl = [f for f in card_reports if k in f]
        assert len(card_fl) == 1
        card_fl = card_fl[0]
        resfinder_fl = [f for f in rf_reports if k in f]
        assert len(resfinder_fl) == 1 
        resfinder_fl = resfinder_fl[0]
        result_dict[k]["card"] = card_fl
        result_dict[k]["resfinder"] = resfinder_fl

    
### END OF DELETE


In [15]:
class isolate:
    
    def __init__(self, guuid):
        self.guuid = guuid
        self.resfinder_fl = pd.read_csv(result_dict[self.guuid]["resfinder"], "\t")
        self.resfinder_trgs = sorted(list(set([resfinder_linkdict[k] for k in list(self.resfinder_fl["GENE"])])))
        self.card_fl = pd.read_csv(result_dict[self.guuid]["card"], "\t")
        self.card_trgs = sorted(list(set([card_linkdict[k] for k in list(self.card_fl["GENE"])])))
        self.resfinder_otrgs = [resfinder_names[k] for k in self.resfinder_trgs]
        self.card_otrgs = [card_names[k] for k in self.card_trgs]
        print(self.guuid)
        print(self.resfinder_otrgs)
        print(self.card_otrgs)
        print("")


for g in guuids:
    x = isolate(g)
    


fd1489de-3070-4a0e-8904-e2084c39edc6
['aac(3)-IIa_1_X51534', "ant(3'')-Ia_1_X02340", 'aadA5_1_AF137361', 'blaCTX-M-15_1_AY044436', "aac(6')-Ib-cr_1_DQ303918", 'mph(A)_1_D16251', 'mph(A)_2_U36578', 'mdf(A)_1_Y08743', 'catB3_2_U13880', 'sul1_5_EU780013', 'tet(A)_6_AF534183', 'dfrA17_6_AF180469', 'blaOXA-1_1_HQ170510']
['AAC(3)-IIe_ARO42600', 'aadA5_ARO39005', 'CTX-M-15_ARO38278', "AAC(6')-Ib-cr_ARO38947", 'catB3_ARO39110', 'sul1_ARO36549', 'tet(A)_ARO36304', 'dfrA17_ARO39294', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'ugd_ARO40187', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'mdtM_ARO37613', 'mphA_ARO36455', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_ARO37208', 'acrD_ARO36630', 'AcrF_ARO36641', 'mdtN_ARO40150', 'AcrE_ARO36638', 'MdtK_ARO37726', 'emrA_ARO360

6f261171-af48-46f2-8c7e-3f2f52137b6c
['blaTEM-1D_1_AF188200', "ant(3'')-Ia_1_X02340", 'mdf(A)_1_Y08743', 'tet(34)_1_AB061440', 'tet(B)_2_AF326777', 'dfrA17_6_AF180469']
['TEM-34_ARO37284', 'dfrA17_ARO39294', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'tet(B)_ARO36305', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'ugd_ARO40187', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_ARO37208', 'acrD_ARO36630', 'AcrF_ARO36641', 'mdtN_ARO40150', 'AcrE_ARO36638', 'MdtK_ARO37726', 'emrA_ARO36036', 'bacA_ARO39420', 'emrB_ARO36213', 'mdtE_ARO37175', 'marA_ARO36402', 'mdtO_ARO40151', "AAC(6')-Ib7_ARO38978", 'Escherichia coli ampC1 beta-lactamase_ARO42575', 'PmrF_ARO40188', 'Escherichia coli ampH beta-lactamase_ARO42576', 'gadW_ARO40529', 'kdpE_ARO40534', 'msbA_ARO40720', 'YojI_

e7b0d544-eb74-41b3-9cfb-93dc7403b5c1
['blaTEM-1D_1_AF188200', "ant(3'')-Ia_1_X02340", "aph(3'')-Ib_5_AF321551", 'mdf(A)_1_Y08743', 'sul2_21_AB366440', 'tet(B)_2_AF326777', 'dfrA17_6_AF180469', 'aph(6)-Id_1_M28829']
['TEM-34_ARO37284', 'APH(6)-Id_ARO39060', 'sul2_ARO36551', 'dfrA17_ARO39294', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'tet(B)_ARO36305', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'ugd_ARO40187', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_ARO37208', 'acrD_ARO36630', 'AcrF_ARO36641', 'mdtN_ARO40150', 'AcrE_ARO36638', 'MdtK_ARO37726', 'emrA_ARO36036', 'bacA_ARO39420', 'emrB_ARO36213', 'mdtE_ARO37175', 'marA_ARO36402', 'mdtO_ARO40151', "AAC(6')-Ib7_ARO38978", 'Escherichia coli ampC1 beta-lactamase_ARO42575', 'PmrF_ARO40188', 'Escherichia coli amp

e0acabbb-e362-492e-a0fc-82e9c0db627c
['blaTEM-1D_1_AF188200', "ant(3'')-Ia_1_X02340", 'lnu(G)_1_KX470419', 'mdf(A)_1_Y08743', 'tet(34)_1_AB061440']
['TEM-34_ARO37284', "ANT(3'')-IIa_ARO41195", 'lnuG_ARO41189', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'ugd_ARO40187', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'mdtM_ARO37613', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_ARO37208', 'acrD_ARO36630', 'AcrF_ARO36641', 'mdtN_ARO40150', 'AcrE_ARO36638', 'mphB_ARO36457', 'MdtK_ARO37726', 'emrA_ARO36036', 'bacA_ARO39420', 'emrB_ARO36213', 'mdtE_ARO37175', 'marA_ARO36402', 'mdtO_ARO40151', 'Escherichia coli ampC1 beta-lactamase_ARO42575', 'PmrF_ARO40188', 'Escherichia coli ampH beta-lactamase_ARO42576', 'gadW_ARO40529', 'kdpE_ARO40534', 'msbA_ARO40720', 'YojI_ARO4072

051a523a-0619-4668-ab82-3037adb23ad0
['aac(3)-IIa_1_X51534', "ant(3'')-Ia_1_X02340", 'aadA5_1_AF137361', 'blaCTX-M-15_1_AY044436', "aac(6')-Ib-cr_1_DQ303918", "aph(3'')-Ib_5_AF321551", 'mph(A)_1_D16251', 'mph(A)_2_U36578', 'mdf(A)_1_Y08743', 'catB3_2_U13880', 'sul1_5_EU780013', 'sul2_18_AJ830714', 'tet(A)_6_AF534183', 'dfrA17_6_AF180469', 'aph(6)-Id_1_M28829', 'blaOXA-1_1_HQ170510']
['AAC(3)-IIe_ARO42600', 'aadA5_ARO39005', 'CTX-M-15_ARO38278', "AAC(6')-Ib-cr_ARO38947", 'APH(6)-Id_ARO39060', 'catB3_ARO39110', 'sul1_ARO36549', 'sul2_ARO36551', 'tet(A)_ARO36304', 'dfrA17_ARO39294', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'ugd_ARO40187', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'mdtM_ARO37613', 'mphA_ARO36455', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_A

...

94f6bc98-629f-4681-b752-48fa0578a0c8
['aac(3)-IId_1_EU022314', 'blaTEM-1D_1_AF188200', "aph(3'')-Ib_5_AF321551", 'mdf(A)_1_Y08743', 'sul2_18_AJ830714', 'tet(34)_1_AB061440', 'tet(A)_6_AF534183', 'aph(6)-Id_1_M28829']
['AAC(3)-IId_ARO42605', 'TEM-34_ARO37284', 'APH(6)-Id_ARO39060', 'sul2_ARO36551', 'tet(A)_ARO36304', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'ugd_ARO40187', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_ARO37208', 'acrD_ARO36630', 'AcrF_ARO36641', 'mdtN_ARO40150', 'AcrE_ARO36638', 'mphB_ARO36457', 'MdtK_ARO37726', 'emrA_ARO36036', 'bacA_ARO39420', 'emrB_ARO36213', 'mdtE_ARO37175', 'marA_ARO36402', 'mdtO_ARO40151', 'Escherichia coli ampC1 beta-lactamase_ARO42575', 'PmrF_ARO40188', 'Escherichia coli ampH

ad6b451b-4c81-4815-8cd2-7fa3352e1543
['blaTEM-1D_1_AF188200', "ant(3'')-Ia_1_X02340", 'aadA5_1_AF137361', "aph(3'')-Ib_5_AF321551", 'mdf(A)_1_Y08743', 'sul2_21_AB366440', 'tet(34)_1_AB061440', 'tet(B)_2_AF326777', 'dfrA17_6_AF180469', 'aph(6)-Id_1_M28829']
['TEM-34_ARO37284', 'aadA5_ARO39005', 'APH(6)-Id_ARO39060', 'sul2_ARO36551', 'dfrA17_ARO39294', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'tet(B)_ARO36305', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'ugd_ARO40187', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'mdtM_ARO37613', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_ARO37208', 'acrD_ARO36630', 'AcrF_ARO36641', 'mdtN_ARO40150', 'AcrE_ARO36638', 'mphB_ARO36457', 'MdtK_ARO37726', 'emrA_ARO36036', 'bacA_ARO39420', 'emrB_ARO36213', 'mdtE_ARO37175', 'marA_ARO36402', 'mdtO_ARO40151', "AAC(6')-Ib7_ARO38

e002bfa3-a306-432c-846b-243feeee6bd6
['blaTEM-1D_1_AF188200', 'mdf(A)_1_Y08743']
['TEM-2_ARO37254', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'ugd_ARO40187', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_ARO37208', 'acrD_ARO36630', 'AcrF_ARO36641', 'mdtN_ARO40150', 'AcrE_ARO36638', 'MdtK_ARO37726', 'emrA_ARO36036', 'bacA_ARO39420', 'emrB_ARO36213', 'mdtE_ARO37175', 'marA_ARO36402', 'mdtO_ARO40151', 'Escherichia coli ampC1 beta-lactamase_ARO42575', 'PmrF_ARO40188', 'Escherichia coli ampH beta-lactamase_ARO42576', 'gadW_ARO40529', 'kdpE_ARO40534', 'msbA_ARO40720', 'YojI_ARO40722', 'Escherichia coli emrE_ARO41083', 'Escherichia coli acrA_ARO41090', 'rpoB2_ARO36640', 'Escherichia coli mdfA_ARO37727', 'Escherichia coli am

8fa45c1f-52c6-4aed-9f6e-0fb173bf7374
['aac(3)-IId_1_EU022314', 'blaTEM-1D_1_AF188200', "ant(3'')-Ia_1_X02340", 'aadA5_1_AF137361', "aph(3')-Ia_1_V00359", "aph(3'')-Ib_5_AF321551", 'mef(B)_1_FJ196385', 'mdf(A)_1_Y08743', 'catA1_1_V00622', 'sul2_21_AB366440', 'sul3_2_AJ459418', 'tet(34)_1_AB061440', 'tet(B)_2_AF326777', 'dfrA17_6_AF180469', 'aadA2_1_NC_010870', 'aph(6)-Id_1_M28829']
['AAC(3)-IId_ARO42605', 'TEM-34_ARO37284', 'aadA5_ARO39005', 'APH(6)-Id_ARO39060', 'mef(B)_ARO39681', 'catI_ARO39117', 'sul2_ARO36551', 'sul3_ARO36552', 'dfrA17_ARO39294', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'tet(B)_ARO36305', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'mdtM_ARO37613', 'aadA2_ARO39002', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_ARO37208', 'acrD_ARO36630', 

c523e54d-b036-4974-a66b-2ba9b2c99fc2
['blaTEM-1D_1_AF188200', "ant(3'')-Ia_1_X02340", 'mdf(A)_1_Y08743', 'sul1_5_EU780013', 'dfrA7_5_AJ419170']
['TEM-34_ARO37284', 'sul1_ARO36549', 'dfrA7_ARO39296', 'mdtP_ARO40152', 'gadX_ARO36647', 'mdtF_ARO37176', 'cpxA_ARO37210', 'mdtH_ARO37615', 'mdtA_ARO37172', 'AcrS_ARO37000', 'emrY_ARO36393', 'mdtG_ARO37728', 'emrK_ARO36345', 'eptA_ARO40186', 'mdtB_ARO37173', 'TolC_ARO36376', 'CRP_ARO36657', 'ugd_ARO40187', 'baeS_ARO37209', 'evgA_ARO37212', 'acrB_ARO36355', 'H-NS_ARO37020', 'mdtC_ARO37174', 'evgS_ARO37213', 'emrR_ARO36655', 'baeR_ARO37208', 'acrD_ARO36630', 'AcrF_ARO36641', 'mdtN_ARO40150', 'AcrE_ARO36638', 'MdtK_ARO37726', 'emrA_ARO36036', 'bacA_ARO39420', 'emrB_ARO36213', 'mdtE_ARO37175', 'marA_ARO36402', 'mdtO_ARO40151', "AAC(6')-Ib7_ARO38978", 'Escherichia coli ampC1 beta-lactamase_ARO42575', 'PmrF_ARO40188', 'Escherichia coli ampH beta-lactamase_ARO42576', 'gadW_ARO40529', 'kdpE_ARO40534', 'msbA_ARO40720', 'YojI_ARO40722', 'Escherichia coli

In [16]:
print(card_db.__dict__)

AttributeError: 'dict' object has no attribute '__dict__'